In [ ]:
#One time Code, Used to extract plain text from websites and save them in a folder.
#Very Few websites encountered some 403 errors --> Their related extraction was done manually


from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request
import csv
from bs4.element import Comment
from rake_nltk import Rake
import sys
import RAKE
import operator
import logging
#History extraction from https://archive.org/help/wayback_api.php
import requests

dates = {"AltS":"20170725","ARWar":"20170725","Bees":"20170725","Bombing":"20170725","Evolution":"20170726","NASA":"20170726","Orcas":"20170726","Sangre" : "20170726","SunTzu" : "20170726","Tornado" : "20170725"}
topic= "AltS"
dest = "Ressource" + topic

def tag_visible(element):
    #Think if we also want to exclude 'a' also. Those are hyperlinks that most of the times contain sitemaps, subscribe
    #... But that is not always the rules.. sometimes they can hyperlink inside the text..
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

docs = pd.read_csv(dest + "\\doc-gain.csv")
docs = docs.set_index("urlid")

for i in docs.index:
    url = docs['url'][i]
    date = dates[topic]
    
    try:
    #Get the old url and open it
     response = requests.get("http://archive.org/wayback/available?url=" +url + "&timestamp=" + date)
     json_res = response.json()
     oldurl =  json_res["archived_snapshots"]["closest"]["url"]
    #Get the old url and open it
     html = urllib.request.urlopen(oldurl).read()
     logging.debug("Link " + oldurl + "was opened")
     urltext = text_from_html(html).encode('utf-8').decode('ascii', 'ignore')
    except urllib.error.HTTPError as e:
     logging.warning("HTTP error for "+ i + ":" + url )
     urltext = ""
    except:
     logging.warning("Could not open history of " + i + ":" + url)
     urltext = "" 
    
    f= open(dest + "\\Corpus\\" + i +".txt","w+")
    f.write(urltext)
    f.close()